In [1]:
from keras.applications import VGG16
#Load the VGG model
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

Using TensorFlow backend.


58892288/58889256 [==============================] - 5s 0us/step


In [2]:
from keras.preprocessing.image import ImageDataGenerator

In [13]:
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7f8c3dc452b0> False
<keras.layers.convolutional.Conv2D object at 0x7f8bf500b470> False
<keras.layers.convolutional.Conv2D object at 0x7f8bf50171d0> False
<keras.layers.pooling.MaxPooling2D object at 0x7f8bf5017c88> False
<keras.layers.convolutional.Conv2D object at 0x7f8bf47c5668> False
<keras.layers.convolutional.Conv2D object at 0x7f8bf477a160> False
<keras.layers.pooling.MaxPooling2D object at 0x7f8bf477aa90> False
<keras.layers.convolutional.Conv2D object at 0x7f8bf47950f0> False
<keras.layers.convolutional.Conv2D object at 0x7f8bf47b3ac8> False
<keras.layers.convolutional.Conv2D object at 0x7f8bf4747c18> False
<keras.layers.pooling.MaxPooling2D object at 0x7f8bf4760f60> False
<keras.layers.convolutional.Conv2D object at 0x7f8bf46fb6d8> False
<keras.layers.convolutional.Conv2D object at 0x7f8bf472d390> False
<keras.layers.convolutional.Conv2D object at 0x7f8bf472dc18> False
<keras.layers.pooling.MaxPooling2D object at 0x7f8bf46e6be0>

In [14]:
from keras import models
from keras import layers
from keras import optimizers
 
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_conv)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4, activation='softmax'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dense_5 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 2052      
Total params: 40,932,676
Trainable params: 33,297,412
Non-trainable params: 7,635,264
________________________________________________________

In [15]:
train_dir = '../input/LabelledRice/Labelled'
validation_dir = '../input/LabelledRice/Labelled'

In [16]:
import os
os.listdir('../input/LabelledRice/Labelled')

['LeafBlast', 'BrownSpot', 'Hispa', 'Healthy']

In [24]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)
 
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=16,
    subset='training',
    class_mode='categorical') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_dir, # same directory as training data
    target_size=(224, 224),
    batch_size=10,
    subset='validation',
    class_mode='categorical')

Found 2686 images belonging to 4 classes.
Found 669 images belonging to 4 classes.


In [18]:
print(train_generator.class_indices)

{'BrownSpot': 0, 'Healthy': 1, 'Hispa': 2, 'LeafBlast': 3}


In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])
# Train the model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=30,
      verbose=1)

Epoch 1/30
168/167 [==============================] - 332s 2s/step - loss: 0.7714 - acc: 0.7257
Epoch 2/30
168/167 [==============================] - 341s 2s/step - loss: 0.7547 - acc: 0.7315
Epoch 3/30
168/167 [==============================] - 329s 2s/step - loss: 0.7322 - acc: 0.7432
Epoch 4/30
168/167 [==============================] - 323s 2s/step - loss: 0.7622 - acc: 0.7398
Epoch 5/30
168/167 [==============================] - 322s 2s/step - loss: 0.6897 - acc: 0.7600
Epoch 6/30
168/167 [==============================] - 320s 2s/step - loss: 0.7278 - acc: 0.7442
Epoch 7/30
168/167 [==============================] - 325s 2s/step - loss: 0.7192 - acc: 0.7514
Epoch 8/30
168/167 [==============================] - 324s 2s/step - loss: 0.6860 - acc: 0.7554
Epoch 9/30
168/167 [==============================] - 323s 2s/step - loss: 0.6934 - acc: 0.7417
Epoch 10/30
168/167 [==============================] - 323s 2s/step - loss: 0.7150 - acc: 0.7561
Epoch 11/30
168/167 [==================